In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df_orig = pd.read_csv('IMDB Dataset.csv')
df_orig.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
 df_orig.shape


(50000, 2)

In [9]:
# Data Cleaning part
# Randomly selecting some reviews set of 15000
df= df_orig
df = df.sample(1000)
df.shape


(1000, 2)

In [ ]:
# gathering infoabout the data 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 19473 to 2301
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     15000 non-null  object
 1   sentiment  15000 non-null  object
dtypes: object(2)
memory usage: 351.6+ KB


In [ ]:
# Checking the count of null values
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [10]:
# Replacing sentiment value column in positive and negative as 0 and 1
from  sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['sentiment']= encoder.fit_transform(df['sentiment'])
df.head()
# this has encoded negative as 0 positive as 1

,review,sentiment
40242,"I rented this movie, thinking it looked like a...",0
3478,A routine mystery/thriller concerning a killer...,0
42309,Lance used to get quality support work from Ja...,0
36395,As with a bunch of guys at school we must give...,1
28868,Different film directors from different countr...,1


In [ ]:
# Replacing sentiment value column in positive and negative with and 0 resectively
# df['sentiment'].replace({'positive':1, 'negative':0}), inplace = True)

In [11]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
# Preprocessor function
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

def preprocess_text(text):
    # remove HTML tags
    text = re.sub('<[^>]*>', '', text)
    
    # lowercase all characters
    text = text.lower()
    
    # remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    text_tokens = nltk.word_tokenize(text)
    filtered_text = [word for word in text_tokens if word not in stop_words]
    text = ' '.join(filtered_text)
    
    # perform stemming
    porter = PorterStemmer()
    text_tokens = nltk.word_tokenize(text)
    stemmed_text = [porter.stem(word) for word in text_tokens]
    text = ' '.join(stemmed_text)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df['review'] = df['review'].apply(preprocess_text)

In [14]:
df['review']

40242    rent movi think look like wonder delight histo...
3478     routin mysteri thriller concern killer lurk sw...
42309    lanc use get qualiti support work jame cameron...
36395    bunch guy school must give thumb even grim rip...
28868    differ film director differ countri contribut ...
                               ...                        
39625    rememb see film year ago think bbc would much ...
31600    note mad scientist everywher go turn son genet...
9612     oh god wast preciou time film pathet waaaay ot...
4088     agre horrend movi ever made proud say copi sim...
34048    bubbl beneath surfac showtim good idea actual ...
Name: review, Length: 1000, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X=cv.fit_transform(df['review']).toarray()
X.shape

(1000, 12431)

In [16]:
y=df['sentiment'].values
y.shape

(1000,)

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [18]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [19]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [20]:
gnb.fit(X_train,y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

0.62
[[52 36]
 [40 72]]
0.6666666666666666


In [21]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

0.855
[[75 13]
 [16 96]]
0.8807339449541285


In [22]:
bnb.fit(X_train,y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))

0.86
[[ 72  16]
 [ 12 100]]
0.8620689655172413
